<a href="https://colab.research.google.com/github/alibakh62/DeepFake-Kaggle/blob/master/notebook/colab/DeepFakeDetection_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version 1

Main steps:



*   Videos -> frames (independent images)
*   Pre-trained convnets -> training only the top layer
*   Prediction on video -> Rolling moving average



# Config and Imports

In [2]:
# from google.colab import drive
# drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [9]:
!pip install ipython-autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=cf039dc0f999bdad56babc6685011a8ffd100ff471b4db652645b6e3c7a1f93b
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [40]:
%load_ext autotime
%tensorflow_version 2.x
%matplotlib inline 

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 12.3 ms


In [41]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
#from tensorflow.keras.applications.imagenet_utils import _obtain_input_shape
from tensorflow.keras.utils import get_file

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
# from tensorflow.keras.utils import np_utils
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.optimizers import SGD
import os
from glob import glob
import math
import h5py
import matplotlib
import matplotlib.pyplot as plt
import random
#import cv2
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform
from sklearn.metrics import accuracy_score
from scipy import misc
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import cv2 as cv

time: 28.2 ms


# Models

### Importing pretrained convnets

In [42]:
from tensorflow.keras.applications import Xception

conv_base = Xception(include_top=False)
# conv_base.summary()

time: 2.32 s


In [43]:
# freezing pretrained model
conv_base.trainable = False

# adding top layers
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="selu"))
model.add(Dense(265, activation="selu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_4 (Dense)              (None, 265)               271625    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 266       
Total params: 23,231,547
Trainable params: 2,370,067
Non-trainable params: 20,861,480
_________________________________________________________________
time: 1.45 s


# DataPrep

In [44]:
# configs
MODELS_FOLDER = '/content/drive/My Drive/DeepFake/models'
DATA_FOLDER = '/content/drive/My Drive/DeepFake'
TRAIN_SAMPLE_FOLDER = 'train_sample_videos'
TEST_FOLDER = 'test_videos'
TRAIN_FRAMES_FOLDER = 'train_frames'
VALID_FRAMES_FOLDER = 'valid_frames'
TEST_FRAMES_FOLDER = 'test_frames'


time: 2.02 ms


### Converting videos into images
***

In [0]:
#@title Sample or All
sample = True #@param {type:"boolean"}
no_of_samples = 5 #@param {type:"integer"}

def get_frames(cap, name, dirname, resize=None):
    frameRate = cap.get(5) #frame rate
    count = 0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a different folder
            if resize is not None:
                # image = np.array(tf.image.resize(frame, resize))
                image = cv.resize(frame, resize)
            else:
                image = frame
            if not os.path.isdir(os.path.join(DATA_FOLDER, dirname)):
                os.mkdir(os.path.join(DATA_FOLDER, dirname))
            count += 1
            fn = os.path.splitext(name)[0] + f"_frame{count}.jpg"
            file_path = os.path.join(DATA_FOLDER, dirname, fn)
            cv.imwrite(file_path, image)
    cap.release()

cnt_max = 2e10
if sample:
    cnt_max = no_of_samples

def walkdir(dirpath):
    for root, dirs, files in os.walk(dirpath):
        for name in files:
            if name != 'metadata.json':
                yield os.path.abspath(os.path.join(dirpath, name)), name

filescount = meta_train_df.index.nunique()

cnt = 1
X_train_full, y_train_full = [], []
for path, name in tqdm_notebook(walkdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)), total=filescount):
    if cnt <= cnt_max:
        cap = cv.VideoCapture(path)
        get_frames(cap, name, TRAIN_FRAMES_FOLDER)
        cnt += 1

### Meta data

In [16]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, json_file))
    df = df.T
    return df

train_list = list(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))
test_list = list(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))
json_file = [file for file in train_list if  file.endswith('json')][0]
meta_train_df = get_meta_from_json(TRAIN_SAMPLE_FOLDER)
meta_train_df.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


time: 11.5 s


### Getting train data & label

In [37]:
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import column_or_1d

images = glob(os.path.join(DATA_FOLDER, f"{TRAIN_FRAMES_FOLDER}/*.jpg"))
X_full, y_full = [], []
for i in tqdm_notebook(images):
    frame_name = i.split('/')[-1]
    image_name = frame_name.split("_")[0]
    label = 1
    if meta_train_df.loc[meta_train_df.index == image_name+'.mp4', "label"].values[0] == 'REAL':
        label = 0
    # reading frames
    img = image.load_img(i, target_size=(512, 512, 3))
    img = image.img_to_array(img)
    img = img/255
    X_full.append(img)
    y_full.append(label)

X = np.array(X_full)
y = column_or_1d(y_full)
print(X.shape)
print(y.shape)


(55, 512, 512, 3)
(55,)
time: 2.28 s


In [39]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_valid: {X_valid.shape}, y_valid: {y_valid.shape}")

X_train: (44, 512, 512, 3), y_train: (44,)
X_valid: (11, 512, 512, 3), y_valid: (11,)
time: 109 ms


# Training

In [45]:
# checkpoints
CHECKPOINT = 'xception_v1.hDF5'
weights_path = os.path.join(MODELS_FOLDER, CHECKPOINT)
from keras.callbacks import ModelCheckpoint
ckp = ModelCheckpoint(weights_path, save_best_only=True, monitor='val_loss', mode='min')

time: 128 ms


Using TensorFlow backend.


In [46]:
# compiling the model
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

time: 71.3 ms


In [48]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_valid, y_valid), callbacks=[ckp], batch_size=16)

Train on 44 samples, validate on 11 samples
Epoch 1/200
44/44 [==============================] - 68s 2s/sample - loss: 0.2612 - accuracy: 1.0000


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


KeyboardInterrupt: ignored

time: 1min 7s
